In [1]:
#Import the libraries I'll use.
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import bs4
import json
import re

# Working Area:

In [589]:
#Starting by looking at a specific game page and looking what class is the box info.
r = requests.get("https://en.wikipedia.org/wiki/Double_Dragon_(video_game)")
soups = bs(r.content,"lxml")

info_box = soups.find(class_="infobox hproduct")
info_rows = info_box.select("tr")
for row in info_rows:
    print(row.prettify())

<tr>
 <th class="infobox-above fn" colspan="2" style="font-size:125%;font-style:italic;">
  Double Dragon
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Ddragon_chirashi.jpg">
   <img alt="Ddragon chirashi.jpg" data-file-height="312" data-file-width="239" decoding="async" height="287" src="//upload.wikimedia.org/wikipedia/en/thumb/e/e6/Ddragon_chirashi.jpg/220px-Ddragon_chirashi.jpg" srcset="//upload.wikimedia.org/wikipedia/en/e/e6/Ddragon_chirashi.jpg 1.5x" width="220"/>
  </a>
  <div class="infobox-caption" style="font-size: 95%;">
   American arcade flyer. The characters have their names localized as Hammer and Spike.
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space:nowrap;padding-right:0.65em;">
  <a href="/wiki/Video_game_developer" title="Video game developer">
   Developer(s)
  </a>
 </th>
 <td class="infobox-data">
  <div class="mw-collapsible mw-collapsed" style="text-align: center; font-size:

In [649]:
for elem in info_box.select("td"):
    if elem.find("br"):
        elem = str(elem)
        elem = elem.replace("<br/>","").replace("\n","").strip()
        elem = bs(elem)
        elem = elem.get_text(", ",strip=True).replace("\xa0","").replace("\n","").strip()
        print(elem)

Technōs Japan, Arc System Works, (Master System), Software Creations, (Genesis)
Taito, [2], Tradewest, ,, Nintendo, (NES), Mastertronic, , Tradewest, (PC), [1], Ballistic, (Genesis)
Yoshihisa Kishimoto, Shinichi Saito
Koji KaiShinichi Saito
Hiroshi SatohTomoyasu KogaNari NishimuraHideki Kaneda
Kumiko MukaiMizuho YamaAkemi TasakiMisae NakayaMasato Shiroto


In [657]:
#Creating a function to check for specific conditions in the another function to make the data values
#clean as possible.
def fixing_elems(elem):
    try:
        if len(elem.find_all("a"))==1:
            return elem.get_text(" ",strip=True) #this checks if an "a" element in each key is single - then to return it not in a form of a list like the other conditions.
        elif elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div"):
            return [el.get_text() for el in elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div")] #This Checks for a specific value locked in a nested div and take the text out.
        elif elem.find("ul",{"class":'mw-collapsible-content hlist'}):
            return elem.a.get_text() #This checks for specific text that is inside a div of a div with no class or id.
        elif elem.select("div.plainlist ul li div.plainlist ul li"):
            elem = [elem.get_text() for elem in elem.select("div.plainlist ul li div.plainlist ul li",limit=3)]
            return elem #This checks for specific text that is above an expandeble list button and take only it.
        elif elem.find("li"):
            return [li.get_text().replace("\xa0","").replace("\n","").strip() for li in elem.find_all("li")] #This create lists and clean them from the "li" elements.
        elif elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"):
            return [text.get_text() for text in elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a")] #This split "a" elements that became a long string.
        elif elem.find("b",{"Home versions":""}):
            elem = str(elem)
            elem = elem.replace("Home versions","")
            elem = bs(elem)
            elem = elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","")
            return elem #This delete a specific section of "Home Version" games releases, since I decided to only the first releases.
        elif elem.find("br"):
            elem = str(elem)
            elem = elem.replace("<br/>" , " ").replace("\n","").strip()
            elem = bs(elem)
            elem = elem.get_text(", ",strip=True).replace("\xa0","").replace("\n","").strip()
            return elem #This deletes line breaks and replace it with a space.
        else:
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","").strip() #This return everything else that is fine.
    except:
        if elem.find(class_="mw-collapsible mw-collapsed"):  
            pass #This Check for the specific class (mostly of lists I don't need because they are an addition to the information) and skip it.

In [658]:
def remove_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose() 

In [659]:
#Creating a function that takes a URL & and getting the data by each category to a dictionary.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    
    remove_tags(soups)
    
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index==0:
            games_info["Title"] = row.find("th").get_text(" ",strip=True)
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = fixing_elems(row.find("td"))
            games_info[key] = value
    
    return games_info
get_info_box("https://en.wikipedia.org/wiki/Double_Dragon_(video_game)")

{'Title': 'Double Dragon',
 'Developer(s)': ['Technōs Japan'],
 'Publisher(s)': ['Taito'],
 'Director(s)': 'Yoshihisa Kishimoto Shinichi Saito',
 'Producer(s)': 'Yoshihisa Kishimoto',
 'Designer(s)': 'Koji Kai Shinichi Saito',
 'Programmer(s)': 'Hiroshi Satoh Tomoyasu Koga Nari Nishimura Hideki Kaneda',
 'Artist(s)': 'Kumiko Mukai Mizuho Yama Akemi Tasaki Misae Nakaya Masato Shiroto',
 'Composer(s)': 'Kazunaka Yamane',
 'Series': 'Double Dragon',
 'Platform(s)': ['Arcade'],
 'Release': ['JP: April 22, 1987',
  'NA: April 1987',
  'EU: June 1987',
  'AU: 1987',
  'JP: April 8, 1988',
  'NA: June 24, 1988',
  'EU: 1990'],
 'Genre(s)': "Beat 'em up",
 'Mode(s)': 'Up to 2 players, simultaneous',
 'Arcade system': 'Custom'}

# Finish Project Area:

In [660]:
#Creating a function to check for specific conditions in the another function to make the data values
#clean as possible.
def fixing_elems(elem):
    try:
        if len(elem.find_all("a"))==1:
            return elem.get_text(" ",strip=True) #this checks if an "a" element in each key is single - then to return it not in a form of a list like the other conditions.
        elif elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div"):
            return [el.get_text() for el in elem.select("#mw-content-text > div.mw-parser-output > table.infobox.hproduct > tbody > tr > td > div > div > div")] #This Checks for a specific value locked in a nested div and take the text out.
        elif elem.find("ul",{"class":'mw-collapsible-content hlist'}):
            return elem.a.get_text() #This checks for specific text that is inside a div of a div with no class or id.
        elif elem.select("div.plainlist ul li div.plainlist ul li"):
            elem = [elem.get_text() for elem in elem.select("div.plainlist ul li div.plainlist ul li",limit=3)]
            return elem #This checks for specific text that is above an expandeble list button and take only it.
        elif elem.find("li"):
            return [li.get_text().replace("\xa0","").replace("\n","").strip() for li in elem.find_all("li")] #This create lists and clean them from the "li" elements.
        elif elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a"):
            return [text.get_text() for text in elem.select("#mw-content-text > div.mw-parser-output > table > tbody > tr > td > a")] #This split "a" elements that became a long string.
        elif elem.find("b",{"Home versions":""}):
            elem = str(elem)
            elem = elem.replace("Home versions","")
            elem = bs(elem)
            elem = elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","")
            return elem #This delete a specific section of "Home Version" games releases, since I decided to only the first releases.
        elif elem.find("br"):
            elem = str(elem)
            elem = elem.replace("<br/>" , " ").replace("\n","").strip()
            elem = bs(elem)
            elem = elem.get_text(", ",strip=True).replace("\xa0","").replace("\n","").strip()
            return elem #This deletes line breaks and replace it with a space.
        else:
            return elem.get_text(" ",strip=True).replace("\xa0","").replace("\n","").strip() #This return everything else that is fine.
    except:
        if elem.find(class_="mw-collapsible mw-collapsed"):  
            pass #This Check for the specific class (mostly of lists I don't need because they are an addition to the information) and skip it.

In [661]:
def remove_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose() #This remove all the sup tags.

In [662]:
#Creating a function that takes a URL & and getting the data by each page link and create dictionaries,
#sorted by the the categories as keys and the values as the keys information.
def get_info_box(url):
    r = requests.get(url)
    soups = bs(r.content,"lxml")
    info_box = soups.find(class_="infobox hproduct")
    info_rows = info_box.find_all("tr")
    
    remove_tags(soups)
    
    games_info = {}
    for index, row in enumerate(info_rows):
        if index==0:
            games_info["Title"] = row.find("th").get_text(" ",strip=True)
        elif index == 1: # That is the picture, I won't use it.
            continue 
        else:
            key = row.find("th").get_text()
            value = fixing_elems(row.find("td"))
            games_info[key] = value
    
    return games_info
# get_info_box("https://en.wikipedia.org/wiki/Donkey_Kong_(video_game)")

In [663]:
#Loaded the page I'm going to get all the video games list and iterate on each game to get the data
#with the help of the two functions i created above.
r = requests.get("https://en.wikipedia.org/wiki/List_of_video_games_considered_the_best")
soup_games = bs(r.content,"lxml")
games = soup_games.select(".wikitable.sortable i a")

wiki_path = "https://en.wikipedia.org"

game_info_list = []
for index, game in enumerate(games):
    try:
        relative_path = game["href"]
        full_wiki_path = wiki_path+relative_path
        fixing_elems(soup_games)
        title = game["title"]
        game_info_list.append(get_info_box(full_wiki_path))
        
    except Exception as e: #3 games won't open so I decided to pass them.
        pass

In [664]:
#Cheking the length of the list I created and saw that it has the same length
#without the games number I couldn't iterate.
len(game_info_list)

236

In [665]:
#Looked at the first 10 games to see that everything went well.
game_info_list[10:]

[{'Title': 'Donkey Kong',
  'Developer(s)': ['Nintendo R&D1', 'Ikegami Tsushinki'],
  'Publisher(s)': 'Nintendo',
  'Director(s)': 'Shigeru Miyamoto',
  'Producer(s)': 'Gunpei Yokoi',
  'Designer(s)': ['Shigeru Miyamoto', 'Gunpei Yokoi'],
  'Artist(s)': 'Shigeru Miyamoto',
  'Composer(s)': 'Yukio Kaneoka',
  'Series': ['Donkey Kong', 'Mario'],
  'Platform(s)': 'Arcade',
  'Release': ['JP: July 9, 1981', 'NA: July 31, 1981', 'EU: November 1981'],
  'Genre(s)': 'Platform',
  'Mode(s)': 'Single-player',
  'Arcade system': 'Radar Scope'},
 {'Title': 'Frogger',
  'Developer(s)': 'Konami',
  'Publisher(s)': ['JP/EU: Konami/Sega', 'NA: Sega/Gremlin Industries'],
  'Series': 'Frogger',
  'Platform(s)': 'Arcade',
  'Release': ['JP: June 5, 1981', 'NA: September 1981', 'EU: 1981'],
  'Genre(s)': 'Action',
  'Mode(s)': 'Single-player'},
 {'Title': 'Galaga',
  'Developer(s)': 'Namco',
  'Publisher(s)': ['JP/EU: Namco', 'NA: Midway Manufacturing'],
  'Director(s)': 'Shigeru Yokoyama',
  'Designer(s

In [666]:
#Created a function to save my list as a json file, since it's easier to save dictionaries
#in this file type.
def save(title, data):
    with open(title, "w", encoding="utf8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
        
save("Video Games Data.json", game_info_list)

In [667]:
#Created a function to load my json file.
def load(title):
    with open(title, "r", encoding="utf8") as f:
        return json.load(f)
    
game_info_list = load("Video Games Data.json")

# Clean The Data:

~~Remove references box from the data and the keys.~~

~~remove whitespaces from the data.~~
### Remove (s) from the dictionary keys, special sign such as "./!" etc and strip whitespaces as well.
### Fix release dates.